In [0]:
%sql
create or replace table dev.demo_db.people(
  id INT,
  firstName STRING,
  lastName STRING,
  birthDate STRING
)USING DELTA;

In [0]:
%sql
INSERT OVERWRITE TABLE dev.demo_db.people
select id,fname as firstName,lname as lastName,dob as birthDate
FROM json.`/Volumes/dev/demo_db/vol1/people.json`

In [0]:
%sql
select * from dev.demo_db.people

In [0]:
%sql
delete from dev.demo_db.people where id = 101

In [0]:
%sql
select * from dev.demo_db.people

In [0]:
from delta import DeltaTable

In [0]:

people_dt = DeltaTable.forName(spark,"dev.demo_db.people")

In [0]:
people_dt.delete("id = 104")

In [0]:
%sql
select * from dev.demo_db.people

In [0]:
%python
from pyspark.sql.functions import lit

people_dt.update(
    condition="id = 103",
    set={"firstName": lit("John")}
)

In [0]:
%sql
select * from dev.demo_db.people

In [0]:
source_df = spark.read.format("json").load("/Volumes/dev/demo_db/vol1/people.json")
display(source_df)

In [0]:
people_dt.toDF().display()

Merge

In [0]:
(
    people_dt.alias("t1")
    .merge(
        source_df.alias("t2"),
        "t1.id = t2.id"
    )
    .whenMatchedDelete(condition="t1.firstName='abdul' and t1.lastName='hamid'")
    .whenMatchedUpdate(condition="t1.id=103",set={"t1.firstName": lit("Cena")})
    .whenMatchedUpdate(set={"t1.id":"t2.id","t1.firstName":"t2.fname","t1.lastName":"t2.lname","t1.birthDate":"t2.dob"})
    .whenNotMatchedInsert(values={"t1.id":"t2.id","t1.firstName":"t2.fname","t1.lastName":"t2.lname","t1.birthDate":"t2.dob"})
    .execute()
)

In [0]:
%sql
select * from dev.demo_db.people

In [0]:
%sql
describe history dev.demo_db.people

In [0]:
%sql
select * from dev.demo_db.people version as of 1

In [0]:
%sql
select * from dev.demo_db.people timestamp AS OF '2025-11-16T17:00:09.000+00:00'

In [0]:
%sql
delete from dev.demo_db.people

In [0]:
%sql
select * from dev.demo_db.people

In [0]:
%sql
describe history dev.demo_db.people

In [0]:
%sql
restore dev.demo_db.people to version as of 8

In [0]:
%sql
select * from dev.demo_db.people

In [0]:
%sql
describe history dev.demo_db.people

Using Spark API

In [0]:
spark.read.format("delta").option("versionAsof","1").table("dev.demo_db.people").display()

In [0]:
spark.read.format("delta").option("timestampAsof","2025-11-17T13:49:19.000+00:00").table("dev.demo_db.people").display()

In [0]:
%sql
delete from dev.demo_db.people

In [0]:
%sql
select * from dev.demo_db.people

In [0]:
from delta import DeltaTable

people_dt = DeltaTable.forName(spark,"dev.demo_db.people")

people_dt.restoreToVersion(10)

In [0]:
%sql
select * from dev.demo_db.people